In [ ]:
#웹 크롤링에 사용될 모듈
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from pprint import pprint

#기타
import re
import json
from geopy.geocoders import Nominatim


In [ ]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

api_key= '382b6c78365fb2bce83b5cd327ecbfbb'
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    try:
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        #위도,경도 반환
        return [float(match_first['x']), float(match_first['y'])]
    except:
        return None
m
total_lst=[]
#페이지 선택
for i in range(1,17):
    url=f'https://www.kcp.or.kr/new/psychologistManagement/list.asp?listType=2&page={i}&rows=15&task=ins&cityi=&gui=&searchText='
    driver.get(url)
    sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #이름이 담긴 태그 선택
    name_lst=soup.select('tr>td[class="text-center vm"]:nth-of-type(2)')
    
     #전화번호와 주소가 담긴 태그를 선택
    phone_and_address= soup.select('td>ul>li>small')
    
    #홀수번째 요소에 전화번호, 짝수번째 요소에 주소가 담김
    phone_lst=phone_and_address[0::2]
    address_lst=phone_and_address[1::2]
    
    #리스트를 문자열로 전환후 <tag_name>을 제거하고, strip을 통해 []을 제거한다. 그후 다시 split을 통해
    #,를 경계로 리스트로 만든다
    # phone_lst=(re.sub('<.+?>', '', phone_lst_source, 0).strip().strip('['']')).split(',')
    # address_lst=(re.sub('<.+?>', '', address_lst_source, 0).strip().strip('['']')).split(',')
    
    # json파일형식으로 데이터를 저장하기 위한 루프문
    for j in range(len(name_lst)):
        temp= {}
        temp['공공/민간']='민간'
        temp['기관구분']='상담소'
        temp['기관명']=re.sub('<.+?>', '', str(name_lst[j]), 0).strip().replace(u'\xa0', u' ')
        temp['주소']=re.sub('<.+?>', '', str(address_lst[j]), 0).strip()
        temp['좌표']=addr_to_lat_lon(temp['주소'])
        temp['전화번호']=re.sub('<.+?>', '', str(phone_lst[j]), 0).strip()
        temp['홈페이지']= None
        total_lst.append(temp)

res={}
res['정신건강관련기관'] =total_lst   
#json 파일으로 변환하기
res_json= json.dumps(res,ensure_ascii=False)
pprint(res_json)

#json파일로 저장하기
with open('전국_임상심리사_상담소.json','w',encoding='utf-8') as f:
    f.write(res_json)

WebDriverException: Message: Service /root/.wdm/drivers/chromedriver/linux64/103.0.5060/chromedriver unexpectedly exited. Status code was: 127


In [ ]:
total_lst

[{'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '(주)마음꼭 차심리상담센터 차심리상담센터',
  '주소': '경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원)',
  '좌표': [127.125249531488, 37.4102499478694],
  '전화번호': '031-780-5437',
  '홈페이지': None},
 {'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '(주)마음꼭 차심리상담센터',
  '주소': '경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원)',
  '좌표': [127.125249531488, 37.4102499478694],
  '전화번호': '031-780-5437',
  '홈페이지': None},
 {'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '(주)마음연구소 부설 마음iN심리상담센터',
  '주소': '서울 종로구 종로3길 34 6층',
  '좌표': [126.979481465774, 37.5718747898899],
  '전화번호': '02-722-2883',
  '홈페이지': None},
 {'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '(주)엠브레스마인드심리상담센터송도',
  '주소': '',
  '좌표': None,
  '전화번호': '032-832-2143',
  '홈페이지': None},
 {'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '가능성연구소',
  '주소': '부산시 해운대구 세실로79 부산해운대대교사옥 6층',
  '좌표': [129.176440231179, 35.1716471915662],
  '전화번호': '051-704-8610',
  '홈페이지': None},
 {'공공/민간': '민간',
  '기관구분': '상담소',
  '기관명': '가톨릭대학교 은평성모병원',
  '주소': '서울 은평구 진관동 9

In [ ]:
import requests

def get_ip():
    response = requests.get('https://api64.ipify.org?format=json').json()
    return response["ip"]
def get_location():
    ip_address = get_ip()
    response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()
    location_data = {
        "ip": ip_address,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name"),
        "latitude": response.get('latitude'),
        "longitude": response.get('longitude')
    }
    return location_data


print(get_location())

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=59f71f48-e620-40de-82b6-60c121211e54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>